# Project 4
## By Lucas Teo

In [8]:
!pip install folium
import folium
import requests
import pandas
import numpy as np

arrest_table = pandas.read_csv("https://cmsc320.github.io/files/BPD_Arrests.csv")

arrest_table = arrest_table[pandas.notnull(arrest_table["Location 1"])]

arrest_table["lat"], arrest_table["long"] = arrest_table["Location 1"].str.split(",").str
arrest_table["lat"] = arrest_table["lat"].str.replace("(", "").astype(float)
arrest_table["long"] = arrest_table["long"].str.replace(")", "").astype(float)

arrest_table.head()

You should consider upgrading via the 'c:\users\lucas\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.
c:\users\lucas\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  # This is added back by InteractiveShellApp.init_path()


,arrest,age,race,sex,arrestDate,arrestTime,arrestLocation,incidentOffense,incidentLocation,charge,chargeDescription,district,post,neighborhood,Location 1,lat,long
1,11127013.0,37,B,M,01/01/2011,00:01:00,2000 Wilkens Ave,79-Other,Wilkens Av & S Payson St,1 1425,Reckless Endangerment || Hand Gun Violation,SOUTHERN,934.0,Carrollton Ridge,"(39.2814026274, -76.6483635135)",39.281403,-76.648364
2,11126887.0,46,B,M,01/01/2011,00:01:00,2800 Mayfield Ave,Unknown Offense,NaN,NaN,Unknown Charge,NORTHEASTERN,415.0,Belair-Edison,"(39.3227699160, -76.5735750473)",39.322770,-76.573575
3,11126873.0,50,B,M,01/01/2011,00:04:00,2100 Ashburton St,79-Other,2100 Ashburton St,1 1106,Reg Firearm:Illegal Possession || Hgv,WESTERN,735.0,Panway/Braddish Avenue,"(39.3117196723, -76.6623546313)",39.311720,-76.662355
4,11126968.0,33,B,M,01/01/2011,00:05:00,4000 Wilsby Ave,Unknown Offense,1700 Aliceanna St,NaN,Unknown Charge,NORTHERN,525.0,Pen Lucy,"(39.3382885254, -76.6045667070)",39.338289,-76.604567
5,11127041.0,41,B,M,01/01/2011,00:05:00,2900 Spellman Rd,81-Recovered Property,2900 Spelman Rd,1 1425,Reckless Endangerment || Handgun Violation,SOUTHERN,924.0,Cherry Hill,"(39.2449886230, -76.6273582432)",39.244989,-76.627358


In [27]:
# The original Baltimore map
map_osm = folium.Map(location=[39.29, -76.61], zoom_start=11)
map_osm

In [28]:
# Take a sample of 300 instead of using the whole 100k rows of data
sample = arrest_table.sample(300)
sample.head()

,arrest,age,race,sex,arrestDate,arrestTime,arrestLocation,incidentOffense,incidentLocation,charge,chargeDescription,district,post,neighborhood,Location 1,lat,long
48298,11375322.0,22,B,M,11/26/2011,14:00:00,1700 E North Ave,5A-Burg. Res. (Force),1600 N Broadway,2C0696,Con-Cds Manuf/Dist-Narc || Cds,EASTERN,312.0,South Clifton Park,"(39.3121681543, -76.5948735787)",39.312168,-76.594874
75307,12489525.0,25,U,M,06/03/2012,04:03:00,1300 W Lexington St,Unknown Offense,NaN,NaN,Unknown Charge,WESTERN,712.0,Franklin Square,"(39.2908659367, -76.6386290322)",39.290866,-76.638629
70543,12469625.0,22,B,M,05/01/2012,02:30:00,3400 Garrison St,4C-Agg. Asslt.- Oth.,3800 Egerton Rd,2 5200,Mace/Chem Device-Int/Injure || Assault,NORTHWESTERN,624.0,Dorchester,"(39.3275523705, -76.6825318447)",39.327552,-76.682532
29652,11277401.0,44,W,M,07/22/2011,09:00:00,2000 W Fayette St,87-Narcotics,0 N Smallwood St,4 3550,Cds:Possess-Not Marihuana || Cds Violation,WESTERN,721.0,Penrose/Fayette Street Outreach,"(39.2894352424, -76.6489540510)",39.289435,-76.648954
43246,11351796.0,21,B,M,10/21/2011,15:20:00,700 Lennox St,78-Gambling,700 Lennox St,2 2210,Trespass-Posted Property || Illegal Gambling,CENTRAL,134.0,Reservoir Hill,"(39.3117010758, -76.6294101283)",39.311701,-76.629410


In [10]:
# Here are some helpful functions that can help generate html for the popups.

# Replacing the placeholder values of 'F' and 'M' with 'female' and 'male' for easier reading.
# If the sex is unknown, it will be stated as such. 
def htmlsex(sex): 
    if (sex == 'F'): 
        return 'female'
    elif (sex == 'M'):
        return 'male'
    else: 
        return 'unknown'
    
# Replacing the placeholder values of 'A', 'B', 'I', and 'W' with 'Asian', 'Black', 'American 
# Indian', and 'White' for easier reading. 
# If the race is unknown, it will be ignored (no race will be specified). 
def htmlrace(race): 
    if (race == 'A'): 
        return ' Asian '
    elif (race == 'B'): 
        return ' Black '
    elif (race == 'I'): 
        return ' American Indian '
    elif (race == 'W'): 
        return ' White '
    elif (race == 'H'):
        return ' Hispanic '
    else: 
        return ' '
    
# Setting up the actual html portion of the popup. 
# We will assume that the most useful information is the age/race/sex as well as the
# date and time of arrest and incident offense. 
def htmlall(row): 
    html = '<center><p>' + \
            str(row['age']) + ' year old'+ htmlrace(row['race']) + htmlsex(row['sex']) + \
            '</br>' + \
            'Arrested: ' + str(row['arrestDate']) + ' ' + str(row['arrestTime']) + \
            '</br>' + \
            'Offense: ' + str(row['incidentOffense']) + \
            '</p></center>'
    return html

### Map 1: Marked with Sex

In [15]:
m1 = folium.Map(location=[39.29, -76.61], zoom_start=11)
for idx, row in sample.iterrows():
    if (row['sex'] == 'M'): 
        folium.Marker(
            location=[row['lat'], row['long']],
            popup=htmlall(row),
            icon=folium.Icon(color='blue', icon='warning', prefix='fa')
        ).add_to(m1)
    elif (row['sex'] == 'F'):
        folium.Marker(
            location=[row['lat'], row['long']],
            popup=htmlall(row),
            icon=folium.Icon(color='pink', icon='warning', prefix='fa')
        ).add_to(m1)

m1

Blue makers = male,<br>
Pink makers = female.

In this map, we map our sample of crimes using markers that indicate the sex of the offender. As we can see below, the majority of offenses are committed by males. Just by eyeing on the map, we estimate that about 50 of these crimes are committed by females, which out of our sample size of 300, which is about 16-20% of the crimes commited in this sample. This follows the common public's perception that most criminals are male (historically). This phenomenon, the gender gap in crime, is actually studied in many fields, such as criminology, sociobiology, and even feminist studies. Our map of Baltimore crime data provide evidence that there is an observable difference in crime rates between men and women.

### Map 2: Marked with Race

In [21]:
m2 = folium.Map(location=[39.29, -76.61], zoom_start=11)
total = 0
asian = 0
black = 0
indian = 0
white = 0
unknown = 0

for idx, row in sample.iterrows():
    total += 1
    if (row['race'] == 'A'): 
        asian += 1
        folium.CircleMarker(
            radius=10,
            location=[row['lat'], row['long']],
            popup=htmlall(row),
            color='yellow',
            fill=True).add_to(m2)
    elif (row['race'] == 'B'): 
        black += 1
        folium.CircleMarker(
            radius=10,
            location=[row['lat'], row['long']],
            popup=htmlall(row),
            color='black',
            fill=True).add_to(m2)
    elif (row['race'] == 'I'):
        indian += 1
        folium.CircleMarker(
            radius=10,
            location=[row['lat'], row['long']],
            popup=htmlall(row),
            color='green',
            fill=True).add_to(m2)
    elif (row['race'] == 'W'):
        white += 1
        folium.CircleMarker(
            radius=10,
            location=[row['lat'], row['long']],
            popup=htmlall(row),
            color='grey',
            fill=True).add_to(m2)
    else:
        unknown += 1
        folium.CircleMarker(
            radius=10,
            location=[row['lat'], row['long']],
            popup=htmlall(row),
            color='red',
            fill=True).add_to(m2)
print(asian/total)
print(black/total)
print(indian/total)
print(white/total)
print(unknown/total)
m2

0.0
0.8666666666666667
0.0
0.12333333333333334
0.01


Yellow = Asian ('A')<br>
Black = Black ('B')<br>
Green = American Indian ('I')<br>
Grey = White ('W')<br>
Red = Unknown ('U')<br>

In the second map, we experiment with mapping markers that indicate race. By observation, we can conclude that most of the crimes are commited by black offenders and the next-most is by white offenders. In our sample, we did not have any Asian offender (yellow circle) and Indian American offender (green circle). We have a few offenders with unknown race. This initial observation gets the sensitive conversation going about the relationship between crime and race. According to the U.S. Department of Justice, about 70% of the crimes were commited by white offenders and 26% of the crimes were commited by black offenders in 2019. However, this is be complete one-eighty in our crime data sample.In our sample, we have about 87% Black criminal offenders and 12% White criminal offenders. One potential flaw in this data is with interracial criminals. We know that there has been an increase in interracial marriages. Thus, it is no clear that such interracial criminals fall under which races. Furthermore, we need to keep in mind that this map simply maps race. There could be underlying factors we can look into, perhaps if a certain offense or type of crime is more likely for one race versus another. According to Wikipedia, research suggests that police practices, such as racial profiling, over-policing in areas populated by minorities and black communities (such as Baltimore), and in-group bias may result in disproportionately high numbers of racial minorities among crime suspects. Therefore, this might explain why we had a result that highly differs from the results from the Department of Justice.

### Map 3: Clustering the marked locations (for fun)

In [23]:
from folium.plugins import MarkerCluster
m3 = folium.Map(location=[39.29, -76.61], zoom_start=11)

marker_cluster3 = MarkerCluster().add_to(m3)

for idx, row in sample.iterrows():
    folium.Marker(
        location=[row['lat'], row['long']],
        popup=htmlall(row),
        icon=folium.Icon(color='red', icon='warning', prefix='fa')
    ).add_to(marker_cluster3)

m3

With clustered locations, we can pick out the hubs of crime easily in order to judge spatially where in the Baltimore area the crimes occured. As we zoom in, we see more partitions/clusters. In this sample, the outskirts of Baltimore have less crime, whereas the closer we get to the heart of the Baltimore, the more crimes we see. Regionally, West Baltimore has the highest number of crimes (110) and East Baltimore has the second-highest number (56). The northern part of Baltimore has 37, while the southern part has 10. We can also point to a specific place and analyze the crime around that area. Let's pick the Johns Hopkins Hospital for example. We see there aren't that many crimes near JHH; there are maybe around 5 to 10 crimes in the immediate area of the hospital. This is reassuring to the hospital staff or visitors and we expect this low number because there is a lot of security and foot traffic near hospitals.This map would be most useful for those familiar with the Baltimore region; they can keep in mind which areas to be wary of. With these clusters, we not only see a big-picture view of Baltimore crime, but also small-picture view of localized, more specific regions of Baltimore.

### Map 4: Combination of all makers above

In [25]:
m4 = folium.Map(location=[39.29, -76.61], zoom_start=11)
marker_cluster4 = MarkerCluster().add_to(m4)

for idx, row in sample.iterrows():

    if (row['sex'] == 'M'): 
        sexIcon = 'mars'
    elif (row['sex'] == 'F'):
        sexIcon = 'venus'
    
    if (row['race'] == 'A'):
        raceColor = 'orange'
    elif (row['race'] == 'B'):
        raceColor = 'black'
    elif (row['race'] == 'I'):
        raceColor = 'green'
    elif (row['race'] == 'W'):
        raceColor = 'gray'    
    else:
        raceColor = 'red'
        
    folium.Marker(
        location=[row['lat'], row['long']],
        popup=htmlall(row),
        icon=folium.Icon(color=raceColor, icon=sexIcon, prefix='fa')
    ).add_to(marker_cluster4)
    
m4

Here, we have a map with location clusters, sex indicators, and race indicators.
We use female/male sex symbols for icons on the markers as the sex indicators.
We use orange/black/green/gray/red (A/B/I/W/U) for colors of the markers as race indicators.

This map combines the ideas of all the maps from above. From this map, it seems like no matter the Baltimore region, the crimes are dominated by black males. In general, most crimes are committed by males, but when they are committed by females, they are usually black. On occasion, there is a white female. For further analysis, we could look into Baltimore demographics in order to get proportional numbers, rather than absolute numbers. For example, there may be more black persons committing crimes since the Baltimore population is heavily black. Thus, these are only our initial observations based on these interactive folium maps. Another future direction could be mapping and analyzing the specific offenses to see if the same offense could be related to a location or another feature.